### Alte Patches, Bilder, Heatmaps und Results löschen

In [1]:
import os
import shutil

folders = ['test_picture', 'test_patches', 'heatmaps_output', 'final_results']

for folder in folders:
    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        continue

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

### Bildvorverarbeitung

In [2]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import random
import shutil
import sys

@dataclass
class MultiScaleConfig:
    patch_size: int = 256
    
    # Skalierungsstufen (Teiler der Bildgröße)
    # Wir decken ein breites Spektrum ab.
    scale_divisors: list = field(default_factory=lambda: [1.5, 2.0, 2.5, 3.0, 4.0])
    
    overlap: float = 0.5 
    
    # --- NEUE LOGIK: Relative Mindestgröße ---
    # Wenn True, wird die min_window_size dynamisch pro Bild berechnet.
    use_relative_min_size: bool = True
    
    # Das kleinste Fenster muss mindestens X % der kürzeren Bildseite sein.
    # 0.12 = 12% (Bei 1080p sind das ca. 130px, bei 4K ca. 260px)
    min_relative_factor: float = 0.15
    
    # Fallback: Absolute Untergrenze in Pixeln, unter die wir NIEMALS gehen wollen,
    # selbst wenn das Bild winzig ist (um extremen Pixelmatsch zu vermeiden).
    absolute_pixel_floor: int = 128
    
    debug_view: bool = True

# --- Hilfsfunktionen ---

def get_square_patch(img, cx, cy, size, target_size=256):
    """Schneidet ein Quadrat aus und skaliert es auf target_size."""
    half = int(size // 2)
    x0, y0 = int(cx - half), int(cy - half)
    x1, y1 = int(x0 + size), int(y0 + size)
    h, w = img.shape[:2]
    
    pad_top = max(0, -y0); pad_bottom = max(0, y1 - h)
    pad_left = max(0, -x0); pad_right = max(0, x1 - w)
    
    if any([pad_top, pad_bottom, pad_left, pad_right]):
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0,0,0])
        x0 += pad_left; x1 += pad_left; y0 += pad_top; y1 += pad_top
        patch = img_padded[y0:y1, x0:x1]
    else:
        patch = img[y0:y1, x0:x1]
        
    # Resize (LANCZOS4 ist am besten für Resizing, sowohl hoch als auch runter)
    if patch.shape[0] != target_size or patch.shape[1] != target_size:
        patch = cv2.resize(patch, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
    return patch

def generate_multiscale_patches(img, cfg: MultiScaleConfig):
    """
    Erzeugt Patches mit dynamischer Mindestgröße basierend auf Bildauflösung.
    """
    h, w = img.shape[:2]
    base_size = min(h, w)
    
    # --- 1. Dynamische Berechnung der Mindestgröße ---
    if cfg.use_relative_min_size:
        # Berechne relative Größe (z.B. 12% von 1080p = 129px)
        dynamic_min = int(base_size * cfg.min_relative_factor)
        # Aber gehe niemals unter den absoluten Boden (z.B. 64px)
        limit_size = max(dynamic_min, cfg.absolute_pixel_floor)
    else:
        # Fester Wert, falls relative Logik deaktiviert ist (für Kompatibilität)
        limit_size = 256 # Default-Wert falls nicht anders definiert

    print(f"   Bildgröße: {w}x{h} -> Dyn. Min-Size: {limit_size}px")
    
    all_patches = []
    all_coords = []
    all_scale_ids = []

    for scale_idx, divisor in enumerate(cfg.scale_divisors):
        # Berechne Soll-Größe für diesen Teiler
        calc_size = int(base_size / divisor)
        
        # --- 2. Smart Clamping ---
        # Wenn die berechnete Größe kleiner ist als unser Limit:
        # Statt zu skippen, setzen wir die Größe auf das Limit.
        # Beispiel: Scale 6.0 ergibt 100px, Limit ist 130px -> Wir nehmen 130px.
        win_size = max(calc_size, limit_size)
        
        # Safety Check: Fenster darf nicht größer sein als das Bild selbst
        win_size = min(win_size, base_size)

        stride = max(1, int(win_size * (1 - cfg.overlap)))
        
        count_for_scale = 0
        for y in range(0, h - win_size + stride, stride):
            for x in range(0, w - win_size + stride, stride):
                cx_top = min(x, w - win_size)
                cy_top = min(y, h - win_size)
                
                center_x = cx_top + win_size // 2
                center_y = cy_top + win_size // 2
                
                patch = get_square_patch(img, center_x, center_y, win_size, cfg.patch_size)
                
                all_patches.append(patch)
                all_coords.append((cx_top, cy_top, win_size))
                all_scale_ids.append(scale_idx)
                count_for_scale += 1
        
        # Hinweis ausgeben, wenn Clamping passiert ist
        status = "✅"
        if calc_size < limit_size:
            status = "⚠️ (Clamped)" # Zeigt an, dass wir die Größe künstlich angehoben haben
            
        print(f"   {status} Scale {divisor} (Calc {calc_size}px -> Used {win_size}px): {count_for_scale} Patches.")

    return all_patches, all_coords, all_scale_ids

# --- Hauptfunktion ---

def main():
    # Pfade anpassen!
    input_dir = Path("~/DatenUbuntu/Studium/1. Semester/KI-Projekt/modeltest/pictures").expanduser()
    test_picture_dir = Path("test_picture") 
    output_root = Path("test_patches")    
    
    cfg = MultiScaleConfig()
    
    # Farben für Visualisierung
    colors = [
        (255, 0, 0),    # Riesig (Blau)
        (255, 255, 0),  # Cyan
        (0, 255, 0),    # Grün
        (0, 255, 255),  # Gelb
        (0, 165, 255),  # Orange
        (0, 0, 255)     # Klein (Rot)
    ]

    # Clean Init
    if test_picture_dir.exists(): shutil.rmtree(test_picture_dir)
    if output_root.exists(): shutil.rmtree(output_root)
    
    test_picture_dir.mkdir(parents=True, exist_ok=True)
    out_patches_root = output_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    # Bilder finden
    if not input_dir.exists(): return
    img_files = list(input_dir.glob("*.[jJ][pP][gG]")) + list(input_dir.glob("*.[pP][nN][gG]"))
    if not img_files: return

    try:
        user_input = input(f"Wie viele Bilder aus '{len(img_files)}' zufällig wählen? (Zahl oder 'all'): ")
        num_to_process = len(img_files) if user_input.lower() == 'all' else int(user_input)
    except: num_to_process = 1

    selected_files = random.sample(img_files, min(num_to_process, len(img_files)))
    patch_metadata = []

    print(f"🚀 Starte Smart Multi-Scale Processing...")

    for img_file in selected_files:
        print(f"\nVerarbeite: {img_file.name}")
        shutil.copy2(img_file, test_picture_dir / img_file.name)
        img = cv2.imread(str(img_file))
        if img is None: continue
        
        h_orig, w_orig = img.shape[:2]
        vis_img = img.copy()
        img_patch_dir = out_patches_root / img_file.stem
        img_patch_dir.mkdir(parents=True, exist_ok=True)

        # B) Patches generieren
        patches, coords, scale_ids = generate_multiscale_patches(img, cfg)
        
        # Duplikate filtern? 
        # Optional: Hier könnte man noch exakte Duplikate (gleiche Koordinaten + Größe) rausfiltern,
        # falls durch das Clamping mehrere Scales auf die gleiche Größe fallen.
        # Der Einfachheit halber lassen wir es erstmal so, da unterschiedliche Scales oft 
        # leicht unterschiedliche Strides haben und somit nicht exakt deckungsgleich sind.

        for i, (p, (x, y, s), s_idx) in enumerate(zip(patches, coords, scale_ids)):
            name = f"S{s_idx}_p{i}.jpg"
            cv2.imwrite(str(img_patch_dir / name), p)
            patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{x};{y};{s};{w_orig};{h_orig}")
            
            color = colors[s_idx % len(colors)]
            thickness = max(1, int(s / 200))
            cv2.rectangle(vis_img, (x, y), (x+s, y+s), color, thickness)

        if cfg.debug_view:
            h, w = vis_img.shape[:2]
            sf = 800 / max(h, w)
            res_small = cv2.resize(vis_img, (int(w * sf), int(h * sf)))
            cv2.putText(res_small, f"MinSize: {int(min(h, w)*cfg.min_relative_factor)}px", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow("Smart Multi-Scale", res_small)
            if cv2.waitKey(100) & 0xFF == ord('q'): break

    with open(output_root / "metadata.txt", "w") as f:
        f.write("\n".join(patch_metadata))
    
    cv2.destroyAllWindows()
    print(f"✅ Fertig!")

if __name__ == "__main__":
    main()

🚀 Starte Smart Multi-Scale Processing...

Verarbeite: Passat_P1_frame2510.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Patches.
   ✅ Scale 3.0 (Calc 240px -> Used 240px): 50 Patches.
   ✅ Scale 4.0 (Calc 180px -> Used 180px): 98 Patches.

Verarbeite: Passat_P1_frame50.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Patches.
   ✅ Scale 3.0 (Calc 240px -> Used 240px): 50 Patches.
   ✅ Scale 4.0 (Calc 180px -> Used 180px): 98 Patches.

Verarbeite: Passat_N1_frame940.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Patc

### Test Model

In [3]:
import os

# 1. System-Konfiguration (Unterdrückt Warnungen und behebt Berechtigungsprobleme)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['MPLCONFIGDIR'] = os.path.join(os.getcwd(), "tmp_matplotlib_cache")
if not os.path.exists(os.environ['MPLCONFIGDIR']):
    os.makedirs(os.environ['MPLCONFIGDIR'], exist_ok=True)

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from ipywidgets import interact, IntSlider

# 2. Pfade und Parameter (Basierend auf deiner Struktur)
model_path = 'final_model.keras'
folder_path = 'test_patches/patches'
img_size = (256, 256)

# 3. Modell laden
try:
    model = load_model(model_path)
    print(f"✅ Modell '{model_path}' erfolgreich geladen.")
except Exception as e:
    print(f"❌ Fehler beim Laden des Modells: {e}")
    model = None

# 4. Bilder-Liste erstellen
if os.path.exists(folder_path):
    files = sorted([str(p.relative_to(folder_path)) for p in Path(folder_path).rglob('*') 
                    if p.suffix.lower() in ('.png', '.jpg', '.jpeg', '.tif')])
    print(f"🔍 {len(files)} Bilder in Unterordnern von '{folder_path}' gefunden.")
else:
    print(f"❌ Ordner '{folder_path}' wurde nicht gefunden!")
    files = []

# 5. Anzeige-Funktion für den Slider
def browse_patches(index):
    if not files:
        print("Keine Bilder vorhanden.")
        return

    filename = files[index]
    img_path = os.path.join(folder_path, filename)
    
    # Bild laden (Grayscale + 256x256)
    img = load_img(img_path, target_size=img_size, color_mode='grayscale')
    img_array = img_to_array(img)
    
    img_tensor = np.expand_dims(img_array, axis=0)

    # Vorhersage
    prediction = model.predict(img_tensor, verbose=0)
    
    # Bestimmung der Wahrscheinlichkeit für Klasse 1
    if prediction.shape[-1] == 1:
        prob_1 = float(prediction[0][0])
    else:
        prob_1 = float(prediction[0][1])
    
    label = 1 if prob_1 > 0.5 else 0
    color = 'green' if label == 1 else 'red'

    # Visualisierung
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array.squeeze(), cmap='gray')
    
    title_str = (f"Bild {index+1}/{len(files)}: {filename}\n"
                 f"KLASSE: {label} | Wahrsch. Klasse 1: {prob_1:.4f}")
    
    plt.title(title_str, color=color, fontsize=12, fontweight='bold')
    plt.axis('off')
    plt.show()

# 6. Interaktives Element starten
if files and model:
    print("Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:")
    interact(browse_patches, index=IntSlider(
        min=0, 
        max=len(files)-1, 
        step=1, 
        value=0, 
        description='Patch-Index:',
        layout={'width': '500px'}
    ))

/home/marlon/DatenUbuntu/Github/KI-Projekt/KI-Project-WS2526/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
I0000 00:00:1768041841.988953  277788 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1207 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


✅ Modell 'final_model.keras' erfolgreich geladen.
🔍 19303 Bilder in Unterordnern von 'test_patches/patches' gefunden.
Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:


interactive(children=(IntSlider(value=0, description='Patch-Index:', layout=Layout(width='500px'), max=19302),…

In [4]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
import gc
from tensorflow.keras.models import load_model

# --- 1. GPU & System Setup (Robust) ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
os.environ['MPLCONFIGDIR'] = str(Path.home() / ".matplotlib_cache")
Path(os.environ['MPLCONFIGDIR']).mkdir(exist_ok=True)

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                try:
                    tf.config.experimental.set_memory_growth(gpu, True)
                except:
                    pass
            print(f"✅ GPU Beschleunigung aktiv.")
    except RuntimeError:
        print("ℹ️ GPU Setup übersprungen.")
setup_gpu()

# --- 2. Konfiguration ---
class QRFinalConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    output_dir = Path('final_results')
    
    # NEU: Voting Konfiguration
    # Nur Patches mit Prob > min_vote_prob dürfen "abstimmen"
    min_vote_prob = 0.0 
    
    # Ab welchem Summen-Wert gilt es als QR-Code? 
    # 2.0 bedeutet: Z.B. 3 Patches mit je 0.7 Sicherheit überlappen sich.
    vote_threshold = 7   
    
    start_batch_size = 32 

# --- 3. Die kombinierte Pipeline ---
def run_combined_pipeline():
    cfg = QRFinalConfig()
    cfg.output_dir.mkdir(exist_ok=True)
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return

    model = load_model(cfg.model_path, compile=False)
    
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel_path, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel_path, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Analyse startet (VOTING MODUS) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        h_orig, w_orig = info["h"], info["w"]
        
        patch_list = info["patches"]
        if not patch_list: continue

        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256), interpolation=cv2.INTER_LINEAR)
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue

        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        preds = None
        current_bs = cfg.start_batch_size
        
        while current_bs >= 1:
            try:
                preds = model.predict(input_batch, batch_size=current_bs, verbose=0)
                break 
            except tf.errors.ResourceExhaustedError:
                current_bs //= 2
                gc.collect() 
                if current_bs < 1:
                    print(f"❌ Bild {img_name} ist zu groß für den VRAM.")
                    preds = None
                    break
        
        if preds is None: continue

        # --- HEATMAP REKONSTRUKTION (VOTING/SUMMIERUNG) ---
        # Wir summieren jetzt, daher können Werte > 1.0 entstehen
        heatmap_sum = np.zeros((h_orig, w_orig), dtype=np.float32)
        max_accumulated_score = 0.0
        
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            
            # --- CHANGE 1: Noise Filter ---
            # Nur Patches, die sich halbwegs sicher sind, dürfen zur Summe beitragen.
            # Das verhindert, dass 50 leere Patches mit 0.1 Prob zu einem "Code" werden.
            if prob > cfg.min_vote_prob:
                p = patch_list[i]
                x, y, s = p["x"], p["y"], p["s"]
                
                # --- CHANGE 2: Summation (Accumulator) ---
                heatmap_sum[y:y+s, x:x+s] += prob

        # Maximalen Score im gesamten Bild finden
        max_accumulated_score = np.max(heatmap_sum)

        del input_batch, all_patch_imgs, preds
        gc.collect()

        # --- Thresholding auf Basis der Summe ---
        # Wir suchen Bereiche, wo die Summe > vote_threshold ist
        _, thresh = cv2.threshold(heatmap_sum, cfg.vote_threshold, 255, cv2.THRESH_BINARY)
        thresh_8bit = thresh.astype(np.uint8)
        
        contours, _ = cv2.findContours(thresh_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # --- Visualisierung Normalisieren ---
        # Da heatmap_sum z.B. bis 8.0 gehen kann, müssen wir es für das Bild normalisieren
        heatmap_vis = heatmap_sum.copy()
        if max_accumulated_score > 0:
            # Skaliere den höchsten Wert auf 255
            heatmap_vis = (heatmap_vis / max_accumulated_score) * 255
        
        heatmap_8bit = heatmap_vis.astype(np.uint8)
        heatmap_blurred = cv2.GaussianBlur(heatmap_8bit, (15, 15), 0)
        heatmap_color = cv2.applyColorMap(heatmap_blurred, cv2.COLORMAP_JET)
        
        overlay = cv2.addWeighted(orig_img, 0.6, heatmap_color, 0.4, 0)
        
        found_count = 0
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            # Etwas größere Mindestgröße, da wir Cluster suchen
            if w > 20 and h > 20: 
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 4)
                found_count += 1

        # Status basiert jetzt auf dem akkumulierten Score
        status = "QR GEFUNDEN" if max_accumulated_score >= cfg.vote_threshold else "KEIN TREFFER"
        
        cv2.rectangle(overlay, (0, 0), (overlay.shape[1], 60), (0, 0, 0), -1)
        bs_info = "" if current_bs == cfg.start_batch_size else f"(BS:{current_bs})"
        
        # Anzeige: "Score: 4.5" statt "Prob: 99%"
        cv2.putText(overlay, f"{status} | Vote-Score: {max_accumulated_score:.2f} {bs_info}", 
                    (20, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        cv2.imwrite(str(cfg.output_dir / f"final_{img_name}"), overlay)
        print(f"✅ {img_name}: {status} (Score: {max_accumulated_score:.2f})")

    tf.keras.backend.clear_session()

if __name__ == "__main__":
    run_combined_pipeline()

✅ GPU Beschleunigung aktiv.
--- Analyse startet (VOTING MODUS) ---
✅ Passat_P1_frame2510.jpg: QR GEFUNDEN (Score: 15.06)
✅ Passat_P1_frame50.jpg: QR GEFUNDEN (Score: 16.35)
✅ Passat_N1_frame940.jpg: KEIN TREFFER (Score: 0.39)
✅ CKlasse_Bilderreihe 2_P_IMG_9448_dark.jpg: QR GEFUNDEN (Score: 16.31)
✅ Passat_P3_frame1170.jpg: QR GEFUNDEN (Score: 14.80)
✅ 20251027_164357.jpg: KEIN TREFFER (Score: 5.70)
✅ Golf_VI_N_11.jpg: KEIN TREFFER (Score: 4.69)
✅ Passat_P3_frame2430.jpg: QR GEFUNDEN (Score: 14.99)
✅ Passat_P3_frame750.jpg: QR GEFUNDEN (Score: 14.48)
✅ Seat_P_7.jpg: QR GEFUNDEN (Score: 15.03)
✅ Passat_N3_frame220.jpg: KEIN TREFFER (Score: 2.18)
✅ Passat_N1_frame3150.jpg: KEIN TREFFER (Score: 1.11)
✅ Passat_N1_frame2840.jpg: KEIN TREFFER (Score: 0.41)
✅ Passat_P1_frame2690.jpg: QR GEFUNDEN (Score: 15.30)
✅ Passat_P2_frame440.jpg: QR GEFUNDEN (Score: 13.81)
✅ CKlasse_Bilderreihe 3_P_IMG_9551_rot-8.jpg: QR GEFUNDEN (Score: 16.68)
✅ Passat_P2_frame1190.jpg: QR GEFUNDEN (Score: 14.65)
✅ Pass

## Heatmap

In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- GPU Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
except: pass

class HeatmapConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    output_folder = Path('heatmaps_output')
    batch_size = 32
    min_vote_prob = 0.50
    
    # NEU: Fixer Skalar für die Visualisierung
    FIXED_MAX_SCORE = 25.0 

def generate_fixed_scale_heatmaps():
    cfg = HeatmapConfig()
    cfg.output_folder.mkdir(exist_ok=True)

    if not cfg.metadata_file.exists():
        print(f"❌ {cfg.metadata_file} nicht gefunden!")
        return

    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path)

    with open(cfg.metadata_file, "r") as f:
        data_lines = [line.strip().split(";") for line in f.readlines()]

    images_dict = {}
    for img_name, rel, px, py, ps, w_orig, h_orig in data_lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"rel_path": rel, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Generiere Heatmaps (Skala 0 bis {cfg.FIXED_MAX_SCORE}) ---")

    for img_name, info in images_dict.items():
        h, w = info["h"], info["w"]
        heatmap_sum = np.zeros((h, w), dtype=np.float32)

        print(f"🚀 Verarbeite: {img_name}")
        
        # --- Batch Loading ---
        patch_list = info["patches"]
        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["rel_path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue

        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)
        preds = model.predict(input_batch, batch_size=cfg.batch_size, verbose=0)

        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            if prob > cfg.min_vote_prob:
                p = patch_list[i]
                heatmap_sum[p["y"]:p["y"]+p["s"], p["x"]:p["x"]+p["s"]] += prob

        # --- FIXE SKALIERUNG (0-25) ---
        # Wir teilen durch 25. Wenn Summe = 25 -> Wert 1.0 -> 255 (Rot)
        # Wenn Summe = 50 -> Wert 2.0 -> wird bei 255 abgeschnitten (bleibt Rot)
        heatmap_norm = (heatmap_sum / cfg.FIXED_MAX_SCORE) * 255.0
        heatmap_8bit = np.clip(heatmap_norm, 0, 255).astype(np.uint8)

        # Speichern als Graustufen (Viewer macht die Farben)
        cv2.imwrite(str(cfg.output_folder / f"{Path(img_name).stem}_heatmap.png"), heatmap_8bit)

    print(f"✅ Alle Heatmaps wurden auf Skala 0-{cfg.FIXED_MAX_SCORE} normiert gespeichert.")

if __name__ == "__main__":
    generate_fixed_scale_heatmaps()

⏳ Lade Modell...
--- Starte Heatmap-Generierung (VOTING) ---
🚀 Verarbeite: Passat_P1_frame2510.jpg
🚀 Verarbeite: Passat_P1_frame50.jpg
🚀 Verarbeite: Passat_N1_frame940.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9448_dark.jpg
🚀 Verarbeite: Passat_P3_frame1170.jpg
🚀 Verarbeite: 20251027_164357.jpg
🚀 Verarbeite: Golf_VI_N_11.jpg
🚀 Verarbeite: Passat_P3_frame2430.jpg
🚀 Verarbeite: Passat_P3_frame750.jpg
🚀 Verarbeite: Seat_P_7.jpg
🚀 Verarbeite: Passat_N3_frame220.jpg
🚀 Verarbeite: Passat_N1_frame3150.jpg
🚀 Verarbeite: Passat_N1_frame2840.jpg
🚀 Verarbeite: Passat_P1_frame2690.jpg
🚀 Verarbeite: Passat_P2_frame440.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 3_P_IMG_9551_rot-8.jpg
🚀 Verarbeite: Passat_P2_frame1190.jpg
🚀 Verarbeite: Passat_P3_frame510.jpg
🚀 Verarbeite: Passat_P2_frame530.jpg
🚀 Verarbeite: Passat_N2_frame1770.jpg
🚀 Verarbeite: Focus_P2_frame_2570.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9449_rot-6.jpg
🚀 Verarbeite: Passat_P1_frame1450.jpg
🚀 Verarbeite: Passat_P2_frame3650.jpg
🚀

### Vergleich der erzeugten Ergebnisse

In [ ]:
import cv2
import numpy as np
from pathlib import Path

# --- PFADE ---
path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

# --- DISPLAY CONFIG ---
# Größeres Fenster für bessere Auflösung
TOTAL_WIDTH = 1800  
IMG_HEIGHT = 600    
LEGEND_WIDTH = 120  
FOOTER_HEIGHT = 100 

def create_legend(height, width):
    """Erstellt einen vertikalen Farbverlauf als Legende für Score 0-25"""
    # Gradient von 255 (oben) bis 0 (unten)
    gradient = np.linspace(255, 0, height).astype(np.uint8)
    gradient = np.tile(gradient, (width, 1)).T
    colored = cv2.applyColorMap(gradient, cv2.COLORMAP_JET)
    
    # Beschriftungen hinzufügen
    # Positionen für 0, 5, 10, 15, 20, 25
    steps = [0, 5, 10, 15, 20, 25]
    for step in steps:
        # Map 0..25 zu Pixel-Koordinate (unten=0, oben=height)
        y_pos = int(height - (step / 25.0 * height))
        # Text etwas versetzen damit er nicht abgeschnitten wird
        y_pos = min(max(y_pos, 20), height - 10)
        
        label = f"{step}"
        if step == 25: label += "+"
        
        cv2.line(colored, (0, y_pos), (15, y_pos), (255, 255, 255), 1)
        cv2.putText(colored, label, (20, y_pos + 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
    # Titel der Legende
    cv2.putText(colored, "Score", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    return colored

def show_interactive_evaluation():
    img_files = sorted(list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]")))
    
    if not img_files:
        print("❌ Keine Originalbilder gefunden.")
        return

    # Breite pro Bild berechnen
    single_w = (TOTAL_WIDTH - LEGEND_WIDTH) // 3
    single_h = IMG_HEIGHT

    stats = {"richtig": 0, "falsch": 0, "gesamt": 0}
    
    print("--- Interaktive Evaluation ---")
    
    # Legende einmalig erstellen
    legend_img = create_legend(single_h, LEGEND_WIDTH)

    for idx, img_file in enumerate(img_files):
        stem = img_file.stem
        
        # 1. Laden
        img_orig = cv2.imread(str(img_file))
        
        # Heatmap laden (Graustufen 0-255, entspricht Score 0-25)
        heat_path = path_heatmap / f"{stem}_heatmap.png"
        img_heat_gray = cv2.imread(str(heat_path), cv2.IMREAD_GRAYSCALE) if heat_path.exists() else None
        
        # Finales Bild laden (nur um sicherzugehen, dass es existiert)
        final_path = path_decision / f"final_{img_file.name}"
        if not final_path.exists():
            final_path = path_decision / f"result_{img_file.name}"
        img_final = cv2.imread(str(final_path)) if final_path.exists() else None

        if img_orig is None or img_heat_gray is None or img_final is None:
            continue

        # 2. Resizing & Color Mapping
        res_orig = cv2.resize(img_orig, (single_w, single_h), interpolation=cv2.INTER_LINEAR)
        res_final = cv2.resize(img_final, (single_w, single_h), interpolation=cv2.INTER_LINEAR)
        
        # Heatmap auf die Zielgröße skalieren
        res_heat_gray = cv2.resize(img_heat_gray, (single_w, single_h), interpolation=cv2.INTER_NEAREST)
        # Jetzt einfärben (Blau=0, Rot=25+)
        res_heat_color = cv2.applyColorMap(res_heat_gray, cv2.COLORMAP_JET)

        # 3. Canvas zusammenbauen
        # Oberer Teil: Bilder + Legende
        images_row = np.hstack((res_orig, res_heat_color, res_final))
        top_section = np.hstack((images_row, legend_img))
        
        # Unterer Teil: Schwarzer Footer für Text
        footer = np.zeros((FOOTER_HEIGHT, top_section.shape[1], 3), dtype=np.uint8)
        
        # 4. Canvas zusammenfügen
        full_canvas = np.vstack((top_section, footer))

        # 5. Texte Zeichnen (Header im Bild, Infos im Footer)
        # Header (Innerhalb der Bilder oben)
        headers = [("Original", 20), ("Heatmap (Vote-Density)", single_w + 20), ("Detection Result", single_w*2 + 20)]
        for txt, x in headers:
            # Text mit Schatten für Lesbarkeit
            cv2.putText(full_canvas, txt, (x+2, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 4)
            cv2.putText(full_canvas, txt, (x, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)

        # Footer Text (Status und Statistik)
        # Extrahiere Infos aus dem Dateinamen oder dummy
        status_line = f"Bild {idx+1}/{len(img_files)}: {img_file.name}"
        stats_line = f"Richtig: {stats['richtig']} | Falsch: {stats['falsch']} | Genauigkeit: {(stats['richtig']/max(1, stats['gesamt'])*100):.1f}%"
        ctrl_line = "[J]a (Richtig)  |  [N]ein (Falsch)  |  [Q]uit"

        cv2.putText(full_canvas, status_line, (20, single_h + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (200, 200, 200), 2)
        cv2.putText(full_canvas, stats_line, (20, single_h + 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(full_canvas, ctrl_line, (single_w * 2, single_h + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # 6. Anzeigen
        cv2.imshow("Evaluation Viewer", full_canvas)
        
        # Input Loop
        valid = False
        while not valid:
            key = cv2.waitKey(0) & 0xFF
            if key == ord('q'):
                cv2.destroyAllWindows()
                print_summary(stats)
                return
            elif key == ord('j'):
                stats["richtig"] += 1; stats["gesamt"] += 1; valid = True
            elif key == ord('n'):
                stats["falsch"] += 1; stats["gesamt"] += 1; valid = True

    print_summary(stats)
    cv2.destroyAllWindows()

def print_summary(stats):
    print("\n=== ABSCHLUSS ===")
    print(f"Total: {stats['gesamt']}")
    print(f"Acc:   {(stats['richtig']/max(1, stats['gesamt'])*100):.2f}%")

if __name__ == "__main__":
    show_interactive_evaluation()

--- Interaktive Evaluation gestartet ---
Steuerung: [J] = Richtig erkannt | [N] = Falsch erkannt | [Q] = Beenden
✅ Bild 1: Richtig markiert.
✅ Bild 2: Richtig markiert.
✅ Bild 3: Richtig markiert.
❌ Bild 4: Falsch markiert.
✅ Bild 5: Richtig markiert.
✅ Bild 6: Richtig markiert.
✅ Bild 7: Richtig markiert.
✅ Bild 8: Richtig markiert.
✅ Bild 9: Richtig markiert.
✅ Bild 10: Richtig markiert.
✅ Bild 11: Richtig markiert.
✅ Bild 12: Richtig markiert.
✅ Bild 13: Richtig markiert.
✅ Bild 14: Richtig markiert.
✅ Bild 15: Richtig markiert.
✅ Bild 16: Richtig markiert.
✅ Bild 17: Richtig markiert.
✅ Bild 18: Richtig markiert.
✅ Bild 19: Richtig markiert.
✅ Bild 20: Richtig markiert.
✅ Bild 21: Richtig markiert.
✅ Bild 22: Richtig markiert.
✅ Bild 23: Richtig markiert.
✅ Bild 24: Richtig markiert.
✅ Bild 25: Richtig markiert.
✅ Bild 26: Richtig markiert.
✅ Bild 27: Richtig markiert.
✅ Bild 28: Richtig markiert.
✅ Bild 29: Richtig markiert.
✅ Bild 30: Richtig markiert.
✅ Bild 31: Richtig markiert